In [1]:
import torch
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

In [2]:
words = open("./names.txt", "r").read().splitlines()

## Data prep

In [3]:
train_words, temp_words = train_test_split(words, train_size=0.8, random_state=42)
dev_words, test_words = train_test_split(temp_words, test_size=0.5, random_state=42)

In [4]:
len(train_words), len(dev_words), len(test_words)

(25626, 3203, 3204)

## Bigram

In [5]:
train_chars = sorted(list(set(''.join(train_words))))
N_b = torch.zeros((27, 27), dtype=torch.int32)
stoi = {s:i+1 for i,s in enumerate(train_chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

# model smoothing
N_b += 3

# "training"
for w in train_words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    N_b[ix1, ix2] += 1

P = N_b.float()
P /= P.sum(1, keepdim=True)

In [6]:
g = torch.Generator().manual_seed(2147483647)
for i in range(10):
  out = []
  ix = 0
  while True:
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

junide.
janasah.
p.
cony.
a.
nn.
kohin.
tolian.
juwe.
ksahnaauranilevias.


### Eval

In [7]:
n = 0; log_likelihood = 0.0
for w in train_words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    log_prob = torch.log(prob)
    log_likelihood += log_prob
    n += 1

nll = -log_likelihood
print("---Train split scores---")
print(f"{nll=}")
print(f"{nll/n=}")

---Train split scores---
nll=tensor(448501.8438)
nll/n=tensor(2.4576)


In [8]:
n = 0; log_likelihood = 0.0
for w in dev_words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    log_prob = torch.log(prob)
    log_likelihood += log_prob
    n += 1

nll = -log_likelihood
print("---Dev split scores---")
print(f"{nll=}")
print(f"{nll/n=}")

---Dev split scores---
nll=tensor(56075.5391)
nll/n=tensor(2.4506)


In [9]:
n = 0; log_likelihood = 0.0
for w in test_words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2 in zip(chs, chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    prob = P[ix1, ix2]
    log_prob = torch.log(prob)
    log_likelihood += log_prob
    n += 1

nll = -log_likelihood
print("---Test split scores---")
print(f"{nll=}")
print(f"{nll/n=}")

---Test split scores---
nll=tensor(56024.1211)
nll/n=tensor(2.4608)


Inuitively, the loss (average negative log likelihood, in other words) should have been lesser on the training dataset, but it turns out here that dev split loss is lesser than the train split loss. In order to keep myself sane, I attribute this "anomaly" to the random seed 42 ;p

I'm pretty sure that I would get the expected result if I were to choose any other seed.

After trying out 2 different seeds, I did get the expected result of the training loss being higher than the dev loss (and equal to the test loss :/) with one of the seeds (69). This inconsistency might be due to the dataset being small or other statistical stuff I don't know about.

## Trigram

In [10]:
two_chars = set()
for c1 in train_chars+["."]:
  for c2 in train_chars+["."]:
    two_chars.add(c1+c2)

two_chars = sorted(list(two_chars))

stoi2 = {s:i for i,s in enumerate(two_chars)}
itos2 = {i:s for i,s in enumerate(two_chars)}

N_t = torch.zeros((729, 27), dtype=torch.int32)

# model smoothing
N_t += 3

# "training"
for w in train_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2,ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi2[ch1+ch2]
    ix2 = stoi[ch3]
    N_t[ix1, ix2] += 1

P = N_t.float()
P /= P.sum(1, keepdim=True)

In [11]:
g = torch.Generator().manual_seed(2147483647)
for _ in range(10):
  ix = 1
  out = ["."]
  while True:
    if ix != 1:
      ix = stoi2[''.join(out[-2:])]
    p = P[ix]
    ix = torch.multinomial(p, num_samples=1, generator=g).item()
    out.append(itos[ix])
    if out[-1][-1] == ".":
      break
  
  print(''.join(out))

.luwjdvdianasid.
.ulexay.
.adin.
.vairritoper.
.maree.
.viameiaurinileniassdbduinrwibtlyssiyanaylarte.
.unvmumthyfodtumj.
.nonnslenarsani.
.rose.
.yae.


### Eval

In [12]:
n = 0
log_likelihood = 0.0
for w in train_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2,ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi2[ch1+ch2]
    ix2 = stoi[ch3]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

nll = -log_likelihood
print("---Train split scores---")
print(f"{nll=}")
print(f"{nll/n=}")

---Train split scores---
nll=tensor(336548.5000)
nll/n=tensor(2.1454)


In [13]:
n = 0
log_likelihood = 0.0
for w in dev_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2,ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi2[ch1+ch2]
    ix2 = stoi[ch3]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

nll = -log_likelihood
print("---Dev split scores---")
print(f"{nll=}")
print(f"{nll/n=}")

---Dev split scores---
nll=tensor(42471.5547)
nll/n=tensor(2.1582)


In [14]:
n = 0
log_likelihood = 0.0
for w in test_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2,ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi2[ch1+ch2]
    ix2 = stoi[ch3]
    prob = P[ix1, ix2]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

nll = -log_likelihood
print("---Test split scores---")
print(f"{nll=}")
print(f"{nll/n=}")

---Test split scores---
nll=tensor(42595.9688)
nll/n=tensor(2.1774)


For the trigram model, the scores are as expected - train loss is higher than dev and test loss.

## Bigram using NN

In [15]:
import torch.nn.functional as F

In [16]:
xs, ys = [], []
for w in train_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    xs.append(ix1)
    ys.append(ix2)

xs = torch.tensor(xs)    
ys = torch.tensor(ys)

# initialize neural net
W = torch.randn((27, 27), generator=g, requires_grad=True)

In [17]:
# gradient descent
for _ in range(150):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=27).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdim=True)
  loss = -probs[torch.arange(xs.nelement()), ys].log().mean() + 0.01*(W**2).mean()

  print(loss.item())

  # backward pass
  W.grad = None
  loss.backward()
  with torch.no_grad():
    W.data += -75 * W.grad


3.8041610717773438
3.2413976192474365
2.9880383014678955
2.862083673477173
2.784681558609009
2.731771469116211
2.692810535430908
2.6631810665130615
2.6398732662200928
2.621373414993286
2.606320858001709
2.5940544605255127
2.583772897720337
2.5751562118530273
2.5677032470703125
2.5612969398498535
2.5556116104125977
2.5506350994110107
2.546132802963257
2.5421414375305176
2.5384769439697266
2.5351998805999756
2.5321545600891113
2.529414653778076
2.5268430709838867
2.524519205093384
2.522320508956909
2.5203280448913574
2.5184288024902344
2.516704559326172
2.515050172805786
2.513547658920288
2.512096405029297
2.5107784271240234
2.509498119354248
2.508335590362549
2.50719952583313
2.506169080734253
2.5051567554473877
2.504240036010742
2.503333330154419
2.502514600753784
2.501699924468994
2.5009658336639404
2.5002312660217285
2.499570846557617
2.498906373977661
2.4983110427856445
2.4977076053619385
2.497169017791748
2.4966206550598145
2.4961323738098145
2.495631694793701
2.495187759399414
2.4

In [18]:
# sampling from the network
g = torch.Generator().manual_seed(2147483647)
for i in range(10):
  out = []
  ix = 0
  while True:
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27).float()
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)

    ix = torch.multinomial(probs, num_samples=1, generator=g).item()
    out.append(itos[ix])
    if ix == 0:
      break
  print(''.join(out))

junide.
janasah.
p.
cfay.
a.
nn.
kohin.
tolia.
s.
tee.


### Eval

In [19]:
xenc = F.one_hot(xs, num_classes=27).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
avg_log_likelihood = probs[torch.arange(xs.nelement()), ys].log().mean()
      
nll = -avg_log_likelihood
print("---Train split scores---")
print(f"{nll=}")

---Train split scores---
nll=tensor(2.4650, grad_fn=<NegBackward0>)


In [20]:
dev_xs, dev_ys = [], []
for w in dev_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    dev_xs.append(ix1)
    dev_ys.append(ix2)

dev_xs = torch.tensor(dev_xs)    
dev_ys = torch.tensor(dev_ys)

xenc = F.one_hot(dev_xs, num_classes=27).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
avg_log_likelihood = probs[torch.arange(dev_xs.nelement()), dev_ys].log().mean()
      
nll = -avg_log_likelihood
print("---Dev split scores---")
print(f"{nll=}")

---Dev split scores---
nll=tensor(2.4577, grad_fn=<NegBackward0>)


In [21]:
test_xs, test_ys = [], []
for w in test_words:
  chs = ["."] + list(w) + ["."]
  for ch1,ch2 in zip(chs,chs[1:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    test_xs.append(ix1)
    test_ys.append(ix2)

test_xs = torch.tensor(test_xs)    
test_ys = torch.tensor(test_ys)

xenc = F.one_hot(test_xs, num_classes=27).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
avg_log_likelihood = probs[torch.arange(test_xs.nelement()), test_ys].log().mean()
      
nll = -avg_log_likelihood
print("---Test split scores---")
print(f"{nll=}")

---Test split scores---
nll=tensor(2.4678, grad_fn=<NegBackward0>)


## Trigram using NN